In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Installing python packages

In [ ]:
!pip install selenium #Keep also if working from local
!apt-get update #Comment this row if working from local
!apt install chromium-chromedriver #Comment this row if working from local (but the Chrome Driver should be manually installed)
!cp /usr/lib/chromium-browser/chromedriver /usr/bin #Comment this row if working from local
!pip install azure-storage-blob==2.1.0
!apt-get install unixodbc-dev
!pip install pyodbc

Downloading and installing the odbc driver for SQL Server 2017 directly from the Microsoft website 

In [ ]:
%%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

Importing modules

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import datetime as dt
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import warnings
from azure.storage.blob import BlockBlobService
from azure.storage.blob import ContentSettings
import re
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
warnings.filterwarnings("ignore")
from datetime import datetime
import pytz
import time
import pyodbc

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
#set local timezone and test result
local_tz = pytz.timezone('Europe/Rome')
print(datetime.now(local_tz))

Setting the connection to the Azure SQL Database

In [ ]:
#db Azure variables
server = 'betsinsights.database.windows.net'
database = 'BETS' 
username = 'admin' 
password = 'xxx' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

Configuration

In [ ]:
#Create variables for unique game ID (check the table DIM_TEAMS on the DB to verify if the team exists and its complete name)
team_1 = 'SquadraCasa'
team_2 = 'SquadraTrasferta'
descr_match = 'SquadraCasa-SquadraTrasferta'
date_match = 'Data'
CODICEPARTITA = 'codicepartita'  #inserire codice partita
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
#stats

link0 = (f'https://www.diretta.it/partita/{CODICEPARTITA}/#statistiche-partite;0')
link1 = (f'https://www.diretta.it/partita/{CODICEPARTITA}/#statistiche-partite;1')
link2 = (f'https://www.diretta.it/partita/{CODICEPARTITA}/#statistiche-partite;2')


#comments
link = (f'https://www.diretta.it/partita/{CODICEPARTITA}/#cronaca-live;0')

delay = 15
filename_stats = (f'{descr_match}_{date_match}_STATS.csv')
filename_comments = (f'{descr_match}_{date_match}_COMMENTS.csv')
path_stats = (f'/content/drive/My Drive/STATS/{filename_stats}')
path_comments = (f'/content/drive/My Drive/STATS/{filename_comments}')

Functions

In [ ]:
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#   
def webpage_shot():
    wd0.save_screenshot('screenshot.png')
    %pylab inline
    img=mpimg.imread('/content/screenshot.png')
    imgplot = plt.imshow(img)
    print(wd.title)
    return plt.show()
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#   
#uploading of the file to Azure container
def upload_file_to_Azure(path,filename,c_name):

  block_blob_service = BlockBlobService(account_name='bets', account_key='xxx')
  container_name = c_name

  try:
      block_blob_service.create_blob_from_path(container_name,filename,path,content_settings=ContentSettings(content_type='application/CSV'))
      print(f'{filename} have been successfully imported into the Azure container {container_name}')  
  except Exception as e:
      print(e)
      pass
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#     

#create the id match for the current match
def create_id_match(c,conn,t1,t2,descr_match,dm):

  id_team_1 = ""
  id_team_2 = ""
  id_match = ""
  
  try:
   c.execute("select id_team from dbo.DIM_Teams where team like trim(upper(?))",'%'+ t1 + '%')
   id_team_1 = c.fetchone()
   if id_team_1 is None:
     raise Exception('Unable to decode team1:',t1) 
   else:
     print('id_team_1 is:',id_team_1[0])
  except Exception as e1:
    print(e1)

  try:
   c.execute("select id_team from dbo.DIM_Teams where team like trim(upper(?))",'%'+ t2 + '%')  
   id_team_2 = c.fetchone()
   if id_team_2 is None:
     raise Exception('Unable to decode team2:',t2) 
   else:
     print('id_team_2 is:',id_team_2[0])
  except Exception as e2:
    print(e2)
    
  id_match = str(id_team_1[0]).zfill(6) + str(id_team_2[0]).zfill(6) + date_match 
  
  try:
   c.execute("INSERT INTO DIM_Match (id_match, match, id_team_1, id_team_2, date_match) VALUES (?,?,?,?,?)",id_match,descr_match,id_team_1[0],id_team_2[0],date_match) 
   conn.commit()
  except Exception as e3:
    print(e3)

  return id_match

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
# decode te icon class in each comment row

def decode_icon(s,min):

  result_icon = ''
  
  try:
    icon_descr = s.find_elements_by_css_selector('div.icon-phrase span.icon')[0].get_attribute("class")
    if icon_descr == 'icon whistle' and min == 90:
      result_icon = 'fischio di fine'
    if icon_descr == 'icon whistle' and min == 1:
      result_icon = 'fischio di inizio'
    if icon_descr == 'icon whistle' and min == 45:
      result_icon = 'fine primo tempo'
    if icon_descr == 'icon whistle' and min == 46:
      result_icon = 'inizio secondo tempo'  
    if icon_descr == 'icon yr-card':
      result_icon = 'secondo cartellino giallo e espulsione'
    if icon_descr == 'icon y-card':
      result_icon = 'cartellino giallo'
    if icon_descr == 'icon funfact':
      result_icon = 'informazione'
    if icon_descr == 'icon soccer-ball':
      result_icon = 'goal'
    if icon_descr == 'icon soccer-ball-own':
      result_icon = 'autogol'      
    if icon_descr == 'icon r-card':
      result_icon = 'cartellino rosso'
    if icon_descr == 'icon lineup1':
      result_icon = 'formazione'
    if icon_descr == 'icon substitution':
      result_icon = 'sostituzione'
    if icon_descr == 'icon time':
      result_icon = 'recupero'
    if icon_descr == 'icon injury':
      result_icon = 'infortunio'
    if icon_descr == 'icon corner':
      result_icon = 'angolo'      
  except Exception as e:
    pass
  return result_icon  

Creating an identifier for this match if it doesn't exist

In [ ]:
#Create unique game ID

id_match = ""

try:
 cursor.execute("SELECT id_match from dbo.DIM_Match m inner join dbo.DIM_Teams t1 on m.id_team_1 = t1.id_team and t1.team = ? inner join dbo.DIM_Teams t2 on m.id_team_2 = t2.id_team and t2.team = ? where date_match = ?",(team_1,team_2,date_match))
 id_match = cursor.fetchone() 
 if id_match is None:
  print('id_match has to be created')
  id_match = create_id_match(cursor,cnxn,team_1,team_2,descr_match,date_match)
  print('New id_match is: ',id_match)
 else:
  id_match = id_match[0]
  print('id_match already exists for this match:',id_match)
except Exception as e:
  print(e)

MATCH STATISTICS - EXTRACTION

In [ ]:
tab_stats0 = []
tab_stats1 = []
tab_stats2 =[]
all_stats = []
i = 0
k = 0
c = 0

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.maximize_window()

wd0 = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd0.maximize_window()
wd0.get(link0)

wd1 = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd1.maximize_window()
wd1.get(link1)

wd2 = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd2.maximize_window()
wd2.get(link2)

time.sleep(delay)

webpage_shot()

team_1 = wd0.find_elements_by_css_selector('div.team-text.tname-home a')[0].text
team_2 = wd0.find_elements_by_css_selector('div.team-text.tname-away a')[0].text
info_match = wd0.find_elements_by_css_selector('span.description__country')[0].text
date_match = wd0.find_elements_by_class_name('description__time')[0].text

print(team_1)
print(team_2)
print(info_match)
print(date_match)

status = 'prematch'
while status != 'Finale' and k < 600: 
  wd0.get(link0)
  wd1.get(link1)
  wd2.get(link2)
  time.sleep(delay)
  tms_loading = dt.datetime.now(local_tz)
  try:
    status = wd0.find_elements_by_css_selector('div.info-status.mstat')[0].text
    print(status)
    if status != 'Finale':
      if status == 'Intervallo' and c == 0:
        c = 1
        print('c = 1')
      if status != 'Intervallo' :
        status = int(status.split('- ')[1].split(':')[0]) + 1
        if c == 0 and (status - 45) > 0:
          status = 45
          print('status = 45')
    print(status)
    result = wd0.find_elements_by_css_selector('#event_detail_current_result')[0].text.replace("""\n""","")
    print(result)
  except:
    pass
  try:
    tab_stats0 = wd0.find_element_by_id("tab-statistics-0-statistic")
  except:
    pass
  try:
    tab_stats1 = wd1.find_element_by_id("tab-statistics-1-statistic")
  except:
    pass
  try:
    tab_stats2 = wd2.find_element_by_id("tab-statistics-2-statistic")
  except:
    pass
  try:
    num = len(tab_stats0.find_elements_by_css_selector("div.statRow"))
  except:
    num = 0
  #num1 = len(tab_stats1.find_elements_by_css_selector("div.statRow"))
  print(k)
  print(i)
  k += 1
  i = 0
  try:
    while  i < num:
      try:
        team_1_value0 = tab_stats0.find_elements_by_css_selector('div.statText.statText--homeValue')[i].text
        team_2_value0 = tab_stats0.find_elements_by_css_selector('div.statText.statText--awayValue')[i].text
        stat_name0 = tab_stats0.find_elements_by_css_selector('div.statText.statText--titleValue')[i].text
      except:
        team_1_value0 = None
        team_2_value0 = None
        stat_name0 = None
      try:
        team_1_value1 = tab_stats1.find_elements_by_css_selector('div.statText.statText--homeValue')[i].text
        team_2_value1 = tab_stats1.find_elements_by_css_selector('div.statText.statText--awayValue')[i].text
        stat_name1 = tab_stats1.find_elements_by_css_selector('div.statText.statText--titleValue')[i].text
      except:
        team_1_value1 = None
        team_2_value1 = None
        stat_name1 = None
      try:
        team_1_value2 = tab_stats2.find_elements_by_css_selector('div.statText.statText--homeValue')[i].text
        team_2_value2 = tab_stats2.find_elements_by_css_selector('div.statText.statText--awayValue')[i].text
        stat_name2 = tab_stats2.find_elements_by_css_selector('div.statText.statText--titleValue')[i].text
      except:
        team_1_value2 = None
        team_2_value2 = None
        stat_name2 = None
      # Schermata 90'
      if stat_name0 == 'Possesso Palla':
        team_1_value0 = team_1_value0.replace('%','')
        team_2_value0 = team_2_value0.replace('%','')
      if team_1_value0 != 0 and stat_name0 == 'Possesso Palla':
        team_1_value0 = int(team_1_value0)/100
      if team_2_value0 != 0 and stat_name0 == 'Possesso Palla':
        team_2_value0 = int(team_2_value0)/100
      # Schermata Primo Tempo
      if stat_name1 == 'Possesso Palla':
        team_1_value1 = team_1_value1.replace('%','')
        team_2_value1 = team_2_value1.replace('%','')
      if team_1_value1 != 0 and stat_name1 == 'Possesso Palla':
        team_1_value1 = int(team_1_value1)/100
      if team_2_value1 != 0 and stat_name1 == 'Possesso Palla':
        team_2_value1 = int(team_2_value1)/100
      # Schermata Secondo Tempo
      if stat_name2 == 'Possesso Palla':
        team_1_value2 = team_1_value2.replace('%','')
        team_2_value2 = team_2_value2.replace('%','')
      if team_1_value2 != 0 and stat_name2 == 'Possesso Palla':
        team_1_value2 = int(team_1_value2)/100
      if team_2_value2 != 0 and stat_name2 == 'Possesso Palla':
        team_2_value2 = int(team_2_value2)/100    
      i += 1
      all_stats.append({'ID': id_match,
                        'tms_loading': tms_loading,
                        'team_1': team_1,
                        'team_2': team_2, 
                        'info_match': info_match,
                        'date_match': date_match,
                        'status': status,
                        'result' : result,
                        'stat_name': stat_name0,
                        'team_1_value0': team_1_value0,
                        'team_2_value0':team_2_value0,  
                        'team_1_value1': team_1_value1,
                        'team_2_value1':team_2_value1,                      
                        'team_1_value2': team_1_value2,
                        'team_2_value2':team_2_value2                             
                        })
  except:
    pass

In [ ]:
ds_all_stats= pd.DataFrame(all_stats)
display(ds_all_stats)

MATCH COMMENTS - EXTRACTION

In [ ]:
#waiting for all comments are updated
time.sleep(60)
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.maximize_window()
wd.get(link)
time.sleep(delay)
webpage_shot()

In [ ]:
team_1 = wd.find_elements_by_css_selector('div.team-text.tname-home')[0].text
team_2 = wd.find_elements_by_css_selector('div.team-text.tname-away')[0].text
info_match = wd.find_elements_by_css_selector('span.description__country')[0].text
date_match = wd.find_elements_by_class_name('description__time')[0].text

In [ ]:
all_comments = []

tms_loading = '' 
comment_icon = ''
comment_minute = ''
comment_team = ''
comment_text = ''

i = 0
k = 0
      
try:

  tab_xpath = wd.find_elements_by_xpath('//*[@id="tab-commentary-0-phrase"]')
  for c in tab_xpath:
     tabella = c.find_element_by_tag_name('tbody')

  while comment_icon != 'fischio di inizio' :

     tms_loading = dt.datetime.now()
     row = tabella.find_elements_by_tag_name('tr')[i].text
     
     #minute 
     comment_minute = row.partition('\n')[0]
     comment_minute = comment_minute.replace("'","")
     comment_minute = comment_minute.strip()

     minute_part_1 = int(comment_minute.partition('+')[0])
     minute_part_2 = comment_minute.partition('+')[2]
     if minute_part_2 == '':
      minute_part_2 = 0
     minute_part_2 = int(minute_part_2)
     if comment_icon == 'inizio secondo tempo':
       k = 1
     if k == 1:
      comment_minute = minute_part_1
     else:
       comment_minute = minute_part_1 + minute_part_2

     #icon decoding
     
     icon_row = tabella.find_elements_by_tag_name('tr')[i]
     comment_icon = decode_icon(icon_row,minute_part_1)

     #comment
     comment_text = row.partition('\n')[2]
     comment_text = comment_text.strip()

     #comment related to one of a team
     comment_team = re.search('\((.*?)\)',comment_text,re.IGNORECASE)
     if comment_team is None:
       comment_team =''
     else:
       comment_team = comment_team.group()
       comment_team = comment_team.strip('()')

     i += 1
     
     all_comments.append({'id': id_match,
                          'tms_loading':tms_loading,
                          'info_match': info_match,
                          'comment_minute': comment_minute,
                          'comment_icon': comment_icon,
                          'comment_team':comment_team,
                          'comment_text': comment_text
                          })
except Exception as e:
  print(e)

In [ ]:
ds_all_comments= pd.DataFrame(all_comments)
display(ds_all_comments)

Preparing csv

In [ ]:
ds_all_stats.to_csv(f'{path_stats}',index = False)
ds_all_comments.to_csv(f'{path_comments}',index = False)

Loading to Azure containers

In [ ]:
upload_file_to_Azure(path_stats,filename_stats,'bets-datasets-diretta-stats')
upload_file_to_Azure(path_comments,filename_comments,'bets-datasets-diretta-comment')